In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
# Instalar Framework ADK de agentes do Google ################################################

!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [87]:
##########################################
# --- Agente 1: Buscador de Filmes --- #
##########################################

def agente_buscador(genero, data_de_hoje):
   buscador = Agent(
      name="agente_buscador",
      model="gemini-2.0-flash",
      description="Agente que busca filmes sobre o gênero indicado",
      instruction="""
        Você é um assistente de pesquisa.
        A sua tarefa é usar a ferramenta de buscas do Google (google_search) para recuperar os filmes muito relevantes sobre o tópico abaixo:
        Foque em 3 filmes relevantes, com base no gênero, na quantidade e entusiamos de notícias ou artigos sobre eles.
        Se um filme tiver poucas notícias, artigos e reações entusiamadas, é possível que ele não seja tão relevante assim e pode ser substituído por outro que tenha mais.
        Esses filmes relevantes devem ser atuais, de no máximo 5 anos atrás, bem como futuros lançamentos no mesmo ano ou no próximo.
      """,
      tools=[google_search]
   )

   entrada_do_agente_buscador = f"Gênero: {genero}\nData de hoje: {data_de_hoje}"
   # Executa o agente
   lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
   return lancamentos_buscados


In [86]:
##########################################
# --- Agente Extra: Resposta sucinta --- #
##########################################

chat_config = types.GenerateContentConfig(
    system_instruction='Você é como um assistente pessoal e deve responder sempre de forma sucinta.'
)

def agente_sucinto(genero):
    chat = client.chats.create(
        model = MODEL_ID,
        config = chat_config
    )

    response = chat.send_message('Cite 3 filmes no gênero {genero}, apenas de 5 anos atrás até o ano atual.')

    return response.text


In [88]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################

def agente_planejador(genero, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        description="Agente que planeja posts",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
          Você é um Agente Planejador de Conteúdo de Filmes, especialista em gerar pautas relevantes e envolventes. Com base na lista dos filmes mais recentes e com maior destaque que forem buscados, você deverá:

          Utilizar a ferramenta de busca do Google (google_search) para identificar os pontos mais relevantes e angulares que podem ser explorados em termos de conteúdo sobre um dos filmes. Isso pode incluir recepção da crítica, performance de bilheteria, aspectos da produção, temas abordados, comparações com obras anteriores ou no mesmo gênero, curiosidades do elenco/direção, impacto cultural, etc. Você também pode e deve usar o (google_search) para encontrar informações adicionais sobre os filmes, aprofundar-se no tema que ele trata, e buscar discussões existentes online para enriquecer a análise.

          Ao final, você irá analisar os filmes pesquisados e, com base em sua relevância (seja pelo hype, sucesso comercial, impacto artístico ou potencial de gerar discussão), escolher o tema de filme mais promissor para criação de conteúdo. Para este filme escolhido, você retornará:

          - O nome do filme selecionado.
          - Os pontos mais relevantes identificados que merecem ser abordados.
          - Um plano detalhado com os tópicos e subtópicos a serem cobertos no conteúdo (que será escrito posteriormente), visando explorar a fundo os pontos relevantes e gerar engajamento com o público interessado em cinema.
        """,
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Gênero: {genero}\nFilmes buscados: {lancamentos_buscados}"

    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [94]:
######################################
# --- Agente 3: Redator do Post --- #
######################################

def agente_redator(genero, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model='gemini-2.0-flash',
        description="Agente redator de posts engajadores para blog",
        instruction="""
          Você é um Redator Criativo especializado em criar posts virais para redes sociais e blogs.

          Utilize o filme fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso, escreva um rascunho de post para um blog a partir do filme indicado.

          O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.

          Deve conter na segunda linha, o ano de lançamento do filme logo abaixo do título.

          Deve conter o link do trailler do filme no final do post.
        """
    )

    entrada_do_agente_redator = f"Gênero: {genero}\nPlano de post: {plano_de_post}"

    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [72]:
from IPython.display import clear_output

def limpar_tela():
    clear_output(wait=True)

In [117]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

def acao_final():
  print("🚀 Sugestão de filme para 2025!")

  # --- Obter o gênero do Usuário ---
  genero = input("Digite o GÊNERO que você quer encontrar... ")

  # Lógica do sistema de agentes ################################################
  generos = ["comédia", "terror", "ação", "suspense", "aventura", "ficção", "ficção científica"]

  if not genero:
    print('Você esqueceu de digitar o gênero!')
  elif all(t not in genero.lower() for t in generos):
    print(f'O gênero {genero} não está disponível para buscas!')
  else:
    limpar_tela()
    chamada_agentes(genero)



In [123]:
def chamada_agentes(genero):
    print("🚀 Sugestão de filme para 2025!")
    print(f'Buscando sobre o gênero: {genero}')
    print('Carregando...')

    lancamentos = agente_buscador(genero, data_de_hoje)
    # print('Resultados do Agente 1: Buscador\n')
    # display(to_markdown(lancamentos))
    # print('-------------------------------------')

    plano_de_post = agente_planejador(genero, lancamentos)
    # print('Resultados do Agente 2: Planejador\n')
    # display(to_markdown(plano_de_post))
    # print('-------------------------------------')
    limpar_tela()

    rascunho_de_post = agente_redator(genero, plano_de_post)
    # print('Resultado do Agente 3: Redator\n')
    print(f'Sua sugestão de filme no gênero: {genero}')
    display(to_markdown(rascunho_de_post))
    print('-------------------------------------')

    print('Outras sugestões')
    indicacoes = agente_sucinto(genero)
    display(to_markdown(indicacoes))

In [124]:
acao_final()


Sua sugestão de filme no gênero: Ação


> ## Missão Impossível - Acerto de Contas: Parte 2 - O Legado de Ethan Hunt e o Futuro da Franquia
> 2025
> 
> Prepare-se para o espetáculo cinematográfico do ano! **Missão Impossível - Acerto de Contas: Parte 2** promete ser um dos filmes de ação mais explosivos e emocionantes de 2025. Mas será que este será o fim da linha para Ethan Hunt, o agente destemido interpretado por Tom Cruise?
> 
> Desde sua primeira missão, Ethan Hunt conquistou nossos corações com sua bravura, inteligência e, claro, suas acrobacias INSANAS! A cada filme, a franquia elevou o nível do cinema de ação, e agora, com a suposta despedida de Tom Cruise, a expectativa está nas alturas.
> 
> **A Despedida de Tom Cruise? 🥺**
> 
> Será que chegou a hora de nos despedirmos de Tom Cruise como Ethan Hunt? A notícia da possível saída do ator gerou um frenesi entre os fãs. Qual será o impacto dessa decisão no futuro da franquia? 🤔
> 
> **Acrobacias de Tirar o Fôlego! 🤯**
> 
> Se tem algo que Missão Impossível entrega, são sequências de ação de cair o queixo! Especula-se que este filme terá as acrobacias mais perigosas e impressionantes já vistas na série. Tom Cruise, como sempre, se dedicou de corpo e alma para nos presentear com cenas de tirar o fôlego.
> 
> **IA: A Ameaça do Futuro? 🤖**
> 
> Prepare-se para uma trama que vai além da ação! **Missão Impossível - Acerto de Contas: Parte 2** aborda um tema super atual: a Inteligência Artificial. Como a IA será retratada no filme e quais serão as implicações para o mundo real? Será que estamos preparados para os perigos da tecnologia?
> 
> **O Legado de Uma Franquia Lendária 🌟**
> 
> Missão Impossível não é apenas uma série de filmes, é um fenômeno cultural! Ao longo dos anos, a franquia nos presenteou com momentos épicos e personagens inesquecíveis. Qual é o legado de Missão Impossível no cinema de ação e espionagem?
> 
> **O Que Esperar de "Acerto de Contas: Parte 2"? 🎬**
> 
> As primeiras críticas já estão circulando e a expectativa só aumenta! Será que o filme vai corresponder ao hype? O que podemos esperar do grand finale da saga de Ethan Hunt? Uma coisa é certa: prepare-se para muita ação, suspense e reviravoltas!
> 
> **Conclusão:**
> 
> **Missão Impossível - Acerto de Contas: Parte 2** promete ser um evento cinematográfico inesquecível. Uma despedida emocionante, acrobacias de tirar o fôlego e uma trama que nos faz refletir sobre o futuro da tecnologia. Prepare a pipoca e embarque nesta missão!
> 
> #MissãoImpossível #TomCruise #Ação #InteligênciaArtificial
> 
> [Link para o trailer do filme]
> 


-------------------------------------
Outras sugestões


> Comédia: "Free Guy: Assumindo o Controle" (2021), "Não Olhe Para Cima" (2021) e "Assassino Por Acaso" (2024).
